In [ ]:
import numpy as np
import time
import pandas as pd 
import urllib
from bs4 import BeautifulSoup

# 수집하고자 하는 id 받아오기

In [ ]:
place_id_list = ['1106610041', '1078930793', '36543606'] # <네이버 지도> 크롤링을 통해 얻은 정보

# 크롤링

In [ ]:
result = pd.DataFrame()
count = 1

for place_id in place_id_list:

    # 크롤링
    url = 'https://m.place.naver.com/place/{}/home?entry=ple'.format(place_id) # 링크1
    try:
        with urllib.request.urlopen(url, timeout = 5) as response:
            html = response.read()
            soup = BeautifulSoup(html, 'html.parser')

            try:
                value7 = str(soup.find_all("span", {"class": "_2yqUQ"})[0]).split('>')[1].split('<')[0]                
                value8 = str(soup.find_all("span", {"class": "_3XamX"})[0]).split('>')[1].split('<')[0]                
                value9 = str(soup.find_all("span", {"class": "_3ocDE"})[0]).split('>')[1].split('<')[0]                
                
            except: # 링크2
                time.sleep(3)
                url_2 = 'https://m.place.naver.com/accommodation/{}'.format(place_id)

                with urllib.request.urlopen(url_2, timeout = 5) as response:
                    html = response.read()
                    soup = BeautifulSoup(html, 'html.parser')
                    try:
                        value7 = str(soup.find_all("span", {"class": "_2yqUQ"})[0]).split('>')[1].split('<')[0]                
                        value8 = str(soup.find_all("span", {"class": "_3XamX"})[0]).split('>')[1].split('<')[0]    
                        value9 = str(soup.find_all("span", {"class": "_3ocDE"})[0]).split('>')[1].split('<')[0]                
                    except:
                        print('=== placeid: {}는 url이 옳지 않습니다. ==='.format(place_id))
                        error_place_id.append(place_id)
    except:
        pass


    # append df
    value_df = pd.DataFrame({'id':[place_id], '가맹점명':[value8], '주소': [value7], '카테고리':[value9]})
    result = result.append(value_df, ignore_index=True)
    count += 1

    # print log
    if count % 10 == 0:
        t2 = time.time()
        elapsed_time= round(t2-t1)
        print('=== {}번째 placeid 완료 === 경과시간: {}초'.format(count, elapsed_time))

# save dataframe
result = result.drop_duplicates(subset=['id']).reset_index(drop=True)
print('크롤링완료!!!!')
result